In [ ]:
# Checking the distributions of the review ratings columns
variables_to_plot = list(dataframe1.columns[dataframe1.columns.str.startswith("review_scores") == True])
fig = plt.figure(figsize=(12,8))
for i, var_name in enumerate(variables_to_plot):
    ax = fig.add_subplot(3,3,i+1)
    dataframe1[var_name].hist(bins=10,ax=ax)
    ax.set_title(var_name)
fig.tight_layout()
plt.show()

#out of these we see that most of them are rated quite positive, with the biggest amount around 4-5/5 stars

In [ ]:
# Importing the Paris neighbourhood  GeoJSON file as a dataframe in geopandas
map_dataframe1 = gpd.read_file('C:/Users/anton/Documents/2de_master/Thesis/Code/analysis/Data/2. airbnb_data/Paris/Q1 airbnb_data Paris/neighbourhoods.geojson')
map_dataframe1.drop('neighbourhood_group', axis = 1, inplace = True)
map_dataframe1.head(20)

In [ ]:
#setting the characteristics for the following plot
plt.rcParams.update({'font.size': 30})
plt.rcParams['figure.figsize'] = 280, 280


# Creating a dataframe of listing counts and median price by neighbourhood
neighbourhood_dataframe1 = pd.DataFrame(dataframe1.groupby('neighbourhood').size())
neighbourhood_dataframe1.rename(columns={0: 'number_of_listings'}, inplace=True)
neighbourhood_dataframe1['median_price'] = dataframe1.groupby('neighbourhood').price.median().values
neighbourhood_dataframe1['mean_price'] = dataframe1.groupby('neighbourhood').price.mean().values

# Putting the dataframes together
neighbourhood_map_dataframe1 = map_dataframe1.set_index('neighbourhood').join(neighbourhood_dataframe1)
                  
# Plotting the number of listings in each neighbourhood
fig1, ax1 = plt.subplots(1, figsize=(15, 6)) #deciding on plot size
neighbourhood_map_dataframe1.plot(column='number_of_listings', cmap='Reds', ax=ax1, rasterized=True) 
#rasterized to makes it easier for big data sets + ax1 earlier defined

ax1.axis('off') #disabling the axis components, including axis labels, ticks, and the frame surrounding the plot
ax1.set_title('Amount of listings per neighbourhood in Paris', fontsize=14)
cax1 = fig1.add_axes([0.9, 0.1, 0.03, 0.8]) # Adjusted the position and size as needed for the side axis [left, bottom, width, height]
#ScalarMappable to make a color map + norm, to normalize the coloring within the graph 
sm1 = plt.cm.ScalarMappable(cmap='Reds', norm=plt.Normalize(vmin=0, vmax=9000))
sm1._A = [] # The primary reason for doing this is to provide an empty array that will later be populated with the data range when the plot is created. The colorbar uses this array to decide the color scaling for the colormap.
cbar1 = fig1.colorbar(sm1, cax=cax1) 
plt.show()

# Plotting the median price of listings in each neighbourhood
fig2, ax2 = plt.subplots(1, figsize=(15, 6))
neighbourhood_map_dataframe1.plot(column='median_price', cmap='Reds', ax=ax2)
ax2.axis('off')
ax2.set_title('Median price per neighbourhood in Paris', fontsize=14)
cax2 = fig2.add_axes([0.9, 0.1, 0.03, 0.8])  # Adjusted the position and size as needed for the side axis [left, bottom, width, height]
#ScalarMappable to make a color map + norm, to normalize the coloring within the graph 
sm2 = plt.cm.ScalarMappable(cmap='Reds', norm=plt.Normalize(vmin=min(neighbourhood_map_dataframe1.median_price), vmax=max(neighbourhood_map_dataframe1.median_price))) 
sm2._A = [] # The primary reason for doing this is to provide an empty array that will later be populated with the data range when the plot is created. The colorbar uses this array to decide the color scaling for the colormap.
cbar2 = fig2.colorbar(sm2, cax=cax2)
plt.show()

# Plotting the mean price of listings in each neighbourhood
fig3, ax3 = plt.subplots(1, figsize=(15, 6))
neighbourhood_map_dataframe1.plot(column='mean_price', cmap='Reds', ax=ax3)
ax3.axis('off')
ax3.set_title('Mean price per neighbourhood in Paris', fontsize=14)
cax3 = fig3.add_axes([0.9, 0.1, 0.03, 0.8])  # Adjusted the position and size as needed for the side axis [left, bottom, width, height]
#ScalarMappable to make a color map + norm, to normalize the coloring within the graph 
sm3 = plt.cm.ScalarMappable(cmap='Reds', norm=plt.Normalize(vmin=min(neighbourhood_map_dataframe1.mean_price), vmax=max(neighbourhood_map_dataframe1.mean_price))) 
sm3._A = [] # The primary reason for doing this is to provide an empty array that will later be populated with the data range when the plot is created. The colorbar uses this array to decide the color scaling for the colormap.
cbar3 = fig3.colorbar(sm3, cax=cax3)
plt.show()

print(neighbourhood_dataframe1)

In [ ]:
# Plot number of listings and median price in one bar plot
plt.rcParams['font.size'] = 15
plt.rcParams['figure.figsize'] = 25, 5
selected_columns = ['number_of_listings', 'median_price']
neighbourhood_dataframe1_selected = neighbourhood_dataframe1[selected_columns]
neighbourhood_dataframe1_sorted = neighbourhood_dataframe1_selected.sort_values('median_price', ascending=False)
neighbourhood_dataframe1_sorted.plot( kind= 'bar' , secondary_y= 'median_price' , rot= 90 )
plt.xticks(rotation='vertical')
plt.show()

# Plot number of listings and mean price in one bar plot
plt.rcParams['font.size'] = 15
plt.rcParams['figure.figsize'] = 25, 5
selected_columns = ['number_of_listings', 'mean_price']
neighbourhood_dataframe1_selected = neighbourhood_dataframe1[selected_columns]
neighbourhood_dataframe1_sorted = neighbourhood_dataframe1_selected.sort_values('mean_price', ascending=False)
neighbourhood_dataframe1_sorted.plot( kind= 'bar' , secondary_y= 'mean_price' , rot= 90 )
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
def binary_count_and_price_plot(column_name,alpha=0.05):
    # Assuming dataframe1 is your DataFrame

    # Plot count distribution
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    sns.countplot(x=column_name, data=dataframe1)
    plt.title(f'Count distribution of {column_name}')

    # Plot median price distribution
    plt.subplot(1, 2, 2)
    median_prices = dataframe1.groupby(column_name)['price'].median()
    median_prices.plot(kind='bar', color=['blue', 'orange'])
    plt.title(f'Median price distribution by {column_name}')
    plt.tight_layout()
    plt.show()

    # Calculate t-test for median prices
    true_prices = dataframe1[dataframe1[column_name] == True]['price']
    false_prices = dataframe1[dataframe1[column_name] == False]['price']
    t_stat, p_value = ttest_ind(true_prices, false_prices)
   
    # Display additional information
    percent_split = dataframe1[column_name].value_counts(normalize=True) * 100
    print(f'% Wise Split of {column_name}:\n{percent_split}')
    t_statistic, p_value = ttest_ind(true_prices, false_prices, equal_var=False)  # assuming unequal variances
    print(f'\nP-value for {column_name}: {p_value}')
    
    # Check if p-value is greater than or equal to alpha
    if p_value >= alpha:
        print(f"{column_name} is not statistically significant (p-value >= {alpha})")
        return column_name
    else:
        print(f"{column_name} is statistically significant (p-value < {alpha})")
        return None  # or any other value if needed

superhost = binary_count_and_price_plot('host_is_superhost')
print(dataframe1.host_is_superhost.value_counts(normalize=True))

test2 = binary_count_and_price_plot('host_identity_verified')
print(dataframe1.host_identity_verified.value_counts(normalize=True))

test3 = binary_count_and_price_plot('instant_bookable')
print(dataframe1.instant_bookable.value_counts(normalize=True))

In [ ]:
# Get column names and their indices
column_info = list(enumerate(dataframe1.columns))
# Print column names and their indices
for index, column_name in column_info:
    print(f"Column {index}: {column_name}")
    
#now we know that the amenity columns start from index 51

In [ ]:
#doing the analysis for all the different amenities
plt.rcParams['font.size'] = 14
plt.rcParams['figure.figsize'] = 20, 20
result = [binary_count_and_price_plot(col) for col in dataframe1.iloc[:,51:-2].columns]

In [ ]:
#now we are making a comparison against the rating and seeing if having the amenity might impact the rating

def binary_count_and_rating_plot(column_name,alpha=0.05):
    # Only taking the ones with reviews into consideration
    data = dataframe1[[column_name, 'review_scores_rating']].dropna()

    # Plot count distribution
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    sns.countplot(x=column_name, data=data)
    plt.title(f'Count distribution of {column_name}')

    # Plot median price distribution
    plt.subplot(1, 2, 2)
    rating_mean = data.groupby(column_name)['review_scores_rating'].mean()
    rating_mean.plot(kind='bar', color=['blue', 'orange'])
    plt.title(f'Average rating distribution by {column_name}')
    plt.tight_layout()
    plt.show()

    # Calculate t-test for median prices
    true_ratings = data[data[column_name] == True]['review_scores_rating']
    false_ratings = data[data[column_name] == False]['review_scores_rating']
    t_stat, p_value = ttest_ind(true_ratings, false_ratings)
   
    # Display additional information
    percent_split = data[column_name].value_counts(normalize=True) * 100
    print(f'% Wise Split of {column_name}:\n{percent_split}')
    t_statistic, p_value = ttest_ind(true_ratings, false_ratings, equal_var=False)  # assuming unequal variances
    print(f'\nP-value for {column_name}: {p_value}')
    
    # Check if p-value is greater than or equal to alpha
    if p_value >= alpha:
        print(f"{column_name} is not statistically significant (p-value >= {alpha})")
        return column_name
    else:
        print(f"{column_name} is statistically significant (p-value < {alpha})")
        return None  # or any other value if needed
    
plt.rcParams['font.size'] = 14
plt.rcParams['figure.figsize'] = 20, 20
result = [binary_count_and_rating_plot(col) for col in dataframe1.iloc[:,51:-2].columns]

In [ ]:
from collections import OrderedDict

def collect_pvalues(columns_of_interest):
    all_pvalues = {}

    for column_name in columns_of_interest:
        # Drop rows with missing values in the specified columns
        data = dataframe1[[column_name, 'price']].dropna()

        # Calculate t-test for median prices
        true_prices = data[data[column_name] == True]['price']
        false_prices = data[data[column_name] == False]['price']
        t_statistic, p_value = ttest_ind(true_prices, false_prices, equal_var=False)  # assuming unequal variances
        all_pvalues[column_name] = {'p_value': p_value, 't_statistic': t_statistic}

    return all_pvalues

columns_of_interest = dataframe1.iloc[:, 51:-2].columns.values
result = collect_pvalues(columns_of_interest)
# Extract p-values and create a dictionary

all_pvalues = {}
for column in result:
    p_value = result[column]['p_value']
    if isinstance(p_value, np.float64):
        all_pvalues[column] = p_value
    else:
        all_pvalues[column] = [float(val) for val in p_value]

import math
#P-values of effects of each amanity on price can be visualized using the 'Manhattan-plot' approach: taking the minus log of the P-value:
# -log Pvalues and order
all_pvalues.update({k: -1*np.log(v) for k, v in all_pvalues.items()})
ordereddict = OrderedDict(sorted(all_pvalues.items(), key=lambda t: t[1]))
plt.rcParams['font.size'] = 10
plt.rcParams['figure.figsize'] = 12, 30
plt.barh(list(ordereddict.keys()), list(ordereddict.values()),color='g', height=0.6)
plt.show()

In [ ]:
plt.rcParams['font.size'] = 14
plt.rcParams['figure.figsize'] = 16, 8

print(f"Of the Airbnb hosts that are still listing on the site, the first joined on {min(dataframe1.host_since).strftime('%d %B %Y')}, and the most recent joined on {max(dataframe1.host_since).strftime('%d %B %Y')}.")


dataframe1.set_index('host_since').resample('MS').size().plot(label='Hosts joining Airbnb', color='orange')
dataframe1.set_index('first_review').resample('MS').size().plot(label='Listings getting their first review', color='green')
plt.title('Paris hosts registration on Airbnb, numbers of first reviews per accommodation per month')
plt.legend()
plt.xlim('2008-01-01', '2019-11-30') # Limiting to whole months
plt.xlabel('')
plt.ylabel('')
plt.show()

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose


def decompose_time_series(ts, title=''):
    decomposition = seasonal_decompose(ts)
    trend = decomposition.trend
    seasonal = decomposition.seasonal
    residual = decomposition.resid

    plt.figure(figsize=(12, 8))

    plt.subplot(411)
    plt.plot(ts, label='Original')
    plt.legend(loc='upper left')
    plt.title(f'{title} - Original')

    plt.subplot(412)
    plt.plot(trend, label='Trend')
    plt.legend(loc='upper left')
    plt.title(f'{title} - Trend')

    plt.subplot(413)
    plt.plot(seasonal, label='Seasonal')
    plt.legend(loc='upper left')
    plt.title(f'{title} - Seasonal')

    plt.subplot(414)
    plt.plot(residual, label='Residual')
    plt.legend(loc='upper left')
    plt.title(f'{title} - Residual')

    plt.tight_layout()
    plt.show()
    
# Creating dataframes for time series analysis
ts_host_since = pd.DataFrame(dataframe1.set_index('host_since').resample('MS').size())
ts_first_review = pd.DataFrame(dataframe1.set_index('first_review').resample('MS').size())

# Renaming columns
ts_host_since = ts_host_since.rename(columns={0: 'hosts'})
ts_host_since.index.rename('month', inplace=True)
ts_first_review = ts_first_review.rename(columns={0: 'reviews'})
ts_first_review.index.rename('month', inplace=True)

plt.rcParams['font.size'] = 12
plt.rcParams['figure.figsize'] = 50, 70
decompose_time_series(ts_host_since, title='Number of registrations on Airbnb by month')

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(dataframe1['reviews_per_month'].dropna(), bins=50, kde=True)
plt.xlabel('Reviews Per Month')
plt.ylabel('Frequency')
plt.title('Reviews Per Month Distribution')
plt.show()

In [ ]:
import folium
from folium.plugins import HeatMap

# Create a base map using the mean latitude and longitude values
mean_lat = dataframe1['latitude'].mean()
mean_lon = dataframe1['longitude'].mean()

m = folium.Map(location=[mean_lat, mean_lon], zoom_start=12)

# Create a HeatMap layer with price as the intensity
heat_data = [[row['latitude'], row['longitude'], row['price']] for index, row in dataframe1.iterrows()]
HeatMap(heat_data, radius=15, blur=10).add_to(m)

# Display the map in Jupyter Notebook
m

In [ ]:
# Finding out the neighborhoods that correspond with the rules we've established.
top_6 = (dataframe1.groupby(['neighbourhood'])['id'].count()
        .sort_values(ascending = False).head(6))

# Let's make a list containing all the 'top_6' Series neighborhoods.
desirable_neighborhoods = list(top_6.index)

# Here, we'll perform a group by. It will list all neighborhoods 
# and their property types available. Also, it will count the number of 
# dwellings each residence kind has.
property_types_count = dataframe1.groupby(['neighbourhood','property_type'], as_index = False)['id'].count()
property_types_count = (property_types_count.sort_values(by = 'id', ascending = False)) 

# Let's make a list containing all the 'top_6' Series neighborhoods.
desirable_neighborhoods = list(top_6.index)

# We'll  filter the DF so that it only contains residences registered 
# in the 'desirable_neighborhoods' list.
property_types_count = (property_types_count[property_types_count['neighbourhood']
                                             .isin(desirable_neighborhoods)])

# And that's it! We are set to plot the chart!
import plotly.express as px

# Unfortunately plotly has a bug for the sunburst plot 
# in which it does not recognize the 'id' column for the chart creation. 

# Hence we'll have to make a second column with the same data.
# We are going to call the new column 'Number of Dwellings'
property_types_count['Number of Dwellings'] = property_types_count['id']

# Finally plotting the sunburst chart!
figure = px.sunburst(property_types_count, values = 'Number of Dwellings', 
                     path = ['neighbourhood','property_type'], width = 600, height = 600, 
                     title = 'Dwelling Type Composition')

figure.update_traces(textinfo="label+percent parent")
figure.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Cut the 'review_scores_value' into categories
categories = pd.cut(dataframe1['review_scores_value'].dropna(), 4, labels=['Poor', 'Average', 'Good', 'Excellent'])
dataframe1['Lodging Quality'] = categories

# Filter out residences with no customer evaluation
no_nan = dataframe1[~dataframe1['review_scores_value'].isnull()]

# Set seaborn style
sns.set(style='whitegrid')

# Group by 'Lodging Quality' and calculate the average price for each category
groups = no_nan.groupby('Lodging Quality')['price'].mean()

# Set the figure size
plt.figure(figsize=(8, 6))

# Plot the bar chart with adjusted ylim
chart = groups.plot(kind='bar', ylim=(0, 300), xlabel='', yticks=[], fontsize=12, color=['grey', 'grey', 'darkred', 'grey'])

# Display average prices on top of the bars
for i, price in enumerate(groups):
    plt.text(i, price + 10, f'${price:.2f}', ha='center', va='bottom', fontdict={'size': 12})

# Set the graph title
plt.title('Average Price per Dwelling Quality', fontdict={'size': 15})

# Show the plot
plt.show()

In [ ]:
#showing the distribution between price and amount of bedrooms
figsize = (10, 5)
x = 'bedrooms'

# Create a grid with 2 rows and 1 column
fig = plt.figure(figsize=figsize)
gs = gridspec.GridSpec(1, 1)

# Bar plot for price distribution
ax1 = plt.subplot(gs[0])
sns.barplot(data=dataframe1, x=x, y='price', ax=ax1)
ax1.set_title(f'Distribution of Price based on {x}')

# Calculate the percentage of each bedroom count relative to all listings
bedroom_percentage = dataframe1[x].value_counts(normalize=True) * 100

# Display the percentage of each bedroom count
for bedrooms, percentage in bedroom_percentage.items():
    print(f'The percentage of listings with {bedrooms} bedroom(s) is: {percentage:.2f}%')


plt.tight_layout()
plt.show()

In [ ]:
#showing the distribution between price and amount of beds
figsize = (10, 5)
x = 'beds'

# Create a grid with 2 rows and 1 column
fig = plt.figure(figsize=figsize)
gs = gridspec.GridSpec(1, 1)

# Bar plot for price distribution
ax1 = plt.subplot(gs[0])
sns.barplot(data=dataframe1, x=x, y='price', ax=ax1)
ax1.set_title(f'Distribution of Price based on {x}')

# Calculate the percentage of each bedroom count relative to all listings
bed_percentage = dataframe1[x].value_counts(normalize=True) * 100

# Display the percentage of each bedroom count
for beds, percentage in bed_percentage.items():
    print(f'The percentage of listings with {beds} bed(s) is: {percentage:.2f}%')


plt.tight_layout()
plt.show()

In [ ]:
#showing the distribution between price and amount of bathrooms
figsize = (10, 5)
x = 'bathrooms'

# Create a grid with 2 rows and 1 column
fig = plt.figure(figsize=figsize)
gs = gridspec.GridSpec(1, 1)

# Bar plot for price distribution
ax1 = plt.subplot(gs[0])
sns.barplot(data=dataframe1, x=x, y='price', ax=ax1)
ax1.set_title(f'Distribution of Price based on {x}')

# Calculate the percentage of each bedroom count relative to all listings
bathroom_percentage = dataframe1[x].value_counts(normalize=True) * 100

# Display the percentage of each bedroom count
for bathrooms, percentage in bathroom_percentage.items():
    print(f'The percentage of listings with {bathrooms} bathroom(s) is: {percentage:.2f}%')


plt.tight_layout()
plt.show()

In [ ]:
#showing the distribution between price and amount of bathrooms
figsize = (10, 5)
x = 'room_type'

# Create a grid with 2 rows and 1 column
fig = plt.figure(figsize=figsize)
gs = gridspec.GridSpec(1, 1)

# Bar plot for price distribution
ax1 = plt.subplot(gs[0])
sns.barplot(data=dataframe1, x=x, y='price', ax=ax1)
ax1.set_title(f'Distribution of Price based on {x}')

# Calculate the percentage of each bedroom count relative to all listings
room_type_percentage = dataframe1[x].value_counts(normalize=True) * 100

# Display the percentage of each bedroom count
for room_type, percentage in room_type_percentage.items():
    print(f'The percentage of listings with {room_type} room type(s) is: {percentage:.2f}%')


plt.tight_layout()
plt.show()

In [ ]:
#showing the distribution between price and amount of bathrooms
figsize = (10, 5)
x = 'accommodates'

# Create a grid with 2 rows and 1 column
fig = plt.figure(figsize=figsize)
gs = gridspec.GridSpec(1, 1)

# Bar plot for price distribution
ax1 = plt.subplot(gs[0])
sns.barplot(data=dataframe1, x=x, y='price', ax=ax1)
ax1.set_title(f'Distribution of Price based on {x}')

# Calculate the percentage of each bedroom count relative to all listings
accommodates_percentage = dataframe1[x].value_counts(normalize=True) * 100

# Display the percentage of each bedroom count
for accommodates, percentage in accommodates_percentage.items():
    print(f'The percentage of listings with {accommodates} accommodates is: {percentage:.2f}%')


plt.tight_layout()
plt.show()

In [ ]:
# Calculate the total number of amenities for each listing
dataframe1['total_amenities'] = dataframe1.iloc[:, 51:-1].sum(axis=1)

#showing the distribution between price and amount of bathrooms
figsize = (10, 5)
x = 'total_amenities'

# Create a grid with 2 rows and 1 column
fig = plt.figure(figsize=figsize)
gs = gridspec.GridSpec(1, 1)

# Bar plot for price distribution
ax1 = plt.subplot(gs[0])
sns.barplot(data=dataframe1, x=x, y='price', ax=ax1)
ax1.set_title(f'Distribution of Price based on {x}')

# Calculate the percentage of each bedroom count relative to all listings
total_amenities_percentage = dataframe1[x].value_counts(normalize=True) * 100

# Display the percentage of each bedroom count
for total_amenities, percentage in total_amenities_percentage.items():
    print(f'The percentage of listings with {total_amenities} total_amenities is: {percentage:.2f}%')

plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim
from sklearn.cluster import DBSCAN, KMeans
import matplotlib.pyplot as plt
import os
import folium
from folium.plugins import MarkerCluster

#Trying to avoid the memory leak of kmeans KML
os.environ['OMP_NUM_THREADS'] = '1'
# Set up the map centered around Paris
paris_map = folium.Map(location=[48.8566, 2.3522], zoom_start=12)
# Add cluster centers to the map using MarkerCluster
marker_cluster = MarkerCluster().add_to(paris_map)

#making a new dataframe for the locations of the listings
data = {
    'latitude': dataframe1['latitude'].head(100).tolist(),
    'longitude': dataframe1['longitude'].head(100).tolist(),
}

df = pd.DataFrame(data)

# Step 1: Reverse geocoding to extract postal codes and ranks
geolocator = Nominatim(user_agent="geo_analyzer")

df['location'] = df.apply(lambda row: geolocator.reverse((row['latitude'], row['longitude'])), axis=1)
df['rank'] = df['location'].apply(lambda loc: loc.raw.get('importance', None))

def rank_to_color(rank):
    if rank is not None:
        rank = float(rank)  # Convert rank to float for better precision
        if rank > 0.5:
            return 'darkgreen'
        elif 0.3 < rank <= 0.5:
            return 'green'
        elif 0.1 < rank <= 0.3:
            return 'lightgreen'
        elif 0.05 < rank <= 0.1:
            return 'lightorange'
        elif rank <= 0.05:
            return 'darkorange'
    return 'gray'

# Step 3: KMeans Clustering with Euclidean distance metric
your_desired_clusters = dataframe1['neighbourhood'].nunique()
kmeans = KMeans(n_clusters=your_desired_clusters, n_init=10)
df['cluster_kmeans'] = kmeans.fit_predict(df[['latitude', 'longitude']])

# Calculate average rank for each cluster
cluster_avg_rank = df.groupby('cluster_kmeans')['rank'].mean().to_dict()
df['cluster_avg_rank'] = df['cluster_kmeans'].map(cluster_avg_rank)

# Create a dictionary to store cluster information
clusters_info = {}

# Iterate through each row to populate the clusters_info dictionary
for idx, row in df.iterrows():
    cluster_label = row['cluster_kmeans']
    
    # If cluster_label is not in clusters_info, create an entry
    if cluster_label not in clusters_info:
        clusters_info[cluster_label] = {
            'latitude_sum': row['latitude'],
            'longitude_sum': row['longitude'],
            'avg_rank': cluster_avg_rank[cluster_label],
            'count': 1
        }
    else:
        # Update sum of coordinates, average rank, and count for existing cluster
        clusters_info[cluster_label]['latitude_sum'] += row['latitude']
        clusters_info[cluster_label]['longitude_sum'] += row['longitude']
        clusters_info[cluster_label]['avg_rank'] += cluster_avg_rank[cluster_label]
        clusters_info[cluster_label]['count'] += 1

# Add each cluster's information to the map
for cluster_label, info in clusters_info.items():
    # Calculate centroid (middle point) of the cluster
    centroid_latitude = info['latitude_sum'] / info['count']
    centroid_longitude = info['longitude_sum'] / info['count']
    
    color = rank_to_color(info['avg_rank'] / info['count'])  # Calculate average rank
    folium.CircleMarker(
        location=[centroid_latitude, centroid_longitude],
        radius=10,  # Adjust the radius based on the count
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=f"Avg Rank: {info['avg_rank'] / info['count']:.2f}, Cluster KMeans: {cluster_label}"
    ).add_to(paris_map)
    
# Plotting the cluster centers on the map of Paris
paris_map

#not yet to the point, so not accurate yet